In [1]:
import os, json, numpy as np, matplotlib.pyplot as plt, seaborn as sns, scipy.io

ART = r"D:\LocalUser\42177 Project\artifacts"
models = ["logreg", "svm", "knn", "trees", "boosting", "cnn"]
metrics = ["Accuracy", "Precision", "Recall", "F1"]

# helper to convert MATLAB loaded objects into native Python types
def _mat_to_py(obj):
    if isinstance(obj, np.ndarray):
        # If it's a 1-element array, unwrap it
        if obj.size == 1:
            return _mat_to_py(obj.item())
        # For larger arrays return a nested list processed recursively
        return [_mat_to_py(x) for x in obj.flatten().tolist()]
    if isinstance(obj, bytes):
        try:
            return obj.decode("utf-8")
        except Exception:
            return str(obj)
    if isinstance(obj, (np.generic,)):
        try:
            return obj.item()
        except Exception:
            return obj
    if isinstance(obj, (list, tuple)):
        return [_mat_to_py(x) for x in obj]
    return obj

# Load results (JSON or MAT)
results = {}
for m in models:
    json_path = os.path.join(ART, f"results_{m}_clean.json")
    mat_path  = os.path.join(ART, f"results_{m}_clean.mat")
    if os.path.exists(json_path):
        with open(json_path) as f:
            results[m] = json.load(f)
    elif os.path.exists(mat_path):
        # use loadmat then convert entries robustly
        mat = scipy.io.loadmat(mat_path)
        py = {k: _mat_to_py(v) for k, v in mat.items() if not k.startswith("__")}

        # MATLAB exports often wrap the actual results under a 'results' field.
        # Try to unwrap common wrappers so downstream code can access keys like 'Accuracy'.
        if "results" in py:
            candidate = py["results"]
            # unwrap single-element lists/tuples
            if isinstance(candidate, (list, tuple)) and len(candidate) == 1:
                candidate = candidate[0]
            # if the candidate is a dict, assume it contains the metric fields we need
            if isinstance(candidate, dict):
                results[m] = candidate
            else:
                # otherwise preserve top-level converted mat content but try to merge useful fields
                flat = {}
                # include other top-level keys (e.g. testSecs)
                for k, v in py.items():
                    if k != "results":
                        flat[k] = v
                # if candidate itself is a dict-like container, merge it
                if isinstance(candidate, dict):
                    flat.update(candidate)
                else:
                    # fallback: keep the 'results' entry as-is so user can inspect it
                    flat["results"] = candidate
                results[m] = flat
        else:
            results[m] = py

# Compare metrics
plt.figure(figsize=(8,5))
for metric in metrics:
    vals = [results[m][metric] for m in models if m in results]
    sns.barplot(x=[m.upper() for m in models if m in results], y=vals)
    plt.ylim(0,1)
    plt.title(f"{metric} Comparison Across Models")
    plt.ylabel(metric)
    plt.show()

# Confusion matrix example for one model
res = results["logreg"]
cm = np.asarray(res["ConfusionMatrix"], dtype=int)
labels = res["Labels"]

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues",
            xticklabels=labels, yticklabels=labels)
plt.xlabel("Predicted"); plt.ylabel("True")
plt.title("Logistic Regression – Confusion Matrix (Clean Test)")
plt.tight_layout(); plt.show()


KeyError: 'Accuracy'

<Figure size 800x500 with 0 Axes>